In [1]:
from ps_model import load_model
from test_personsearch_prw_sys import get_prw_dataset_info, get_prw_data, search_performance_prw
import torch
from __init__ import *
info_sota = get_info_sota()

/home/linhuadong/miniconda3/envs/mmdet/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [2]:
PRW_Dataset, pname_to_attribute, gt_roidb, name_to_det_feat_prw = get_prw_dataset_info(info_sota['PRW'])

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
data_infos 1703
probes_name_list 2057


In [ ]:
model = load_model(info_sota['PRW'], 2)

In [ ]:
idx = 2
data = get_prw_data(PRW_Dataset, pname_to_attribute, idx)
with torch.no_grad():
    result = model(return_loss=False, rescale=True, **data)
entry = search_performance_prw(result, data, pname_to_attribute, name_to_det_feat_prw, gt_roidb)

In [ ]:
def visualize(image_root, probe_img, probe_roi, color, save_name):
    file_root = os.path.join(image_root, probe_img)
    img = Image.open(file_root)
    plt.figure(figsize=(8, 8))
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    
    ax = plt.gca()
    down_left_x, down_left_y = probe_roi[0], probe_roi[1]
    width = probe_roi[2] - probe_roi[0]
    height = probe_roi[3] - probe_roi[1]
    rect = plt.Rectangle((down_left_x, down_left_y), width, height, fill=False, edgecolor=color, linewidth=1)
    ax.add_patch(rect)
    save_root_dir = "/home/linhuadong/DNFNet/tools/system/cache"
    if not os.path.exists(save_root_dir):
        os.makedirs(save_root_dir)
    plt.savefig(os.path.join(save_root_dir, save_name))

def search(results):
    colors = ['red', 'green', 'yellow']
    search_result = []
    visualize(results['image_root'], results['probe_img'], results['probe_roi'], colors[2]) # query
    for j in range(5):
        info = results['gallery'][j]
        img = visualize(results['image_root'], info['img'], info['roi'][:4], colors[info['correct']], save_name=f'top-{j+1}')  # gallery
    return True

search(entry)
